In [5]:
import requests
import json
import pandas as pd
import time
import keepa
import numpy as np
api_key = "a1g603bd0m13rt71rksm50t36ane60qc8kvioi1ivo3b4sf1qkbj0net036tt6ql"
api = keepa.Keepa(api_key)


In [6]:
def fetch_products_from_queries(query_series, allowed_brands,dfname='products' ,max_per_minute=20):
    """
    For each search query and allowed brand, fetch 1 product from Keepa API.
    Applies rate-limiting and returns a flattened DataFrame.
    """
    results = []

    for keyword in query_series:
        product_row = {"Search Query": keyword}
        column_suffix = 1

        for brand in allowed_brands:
            product_params = {
                'domain': 1,
                'term': keyword,
                'brand': brand
            }

            try:
                asin_list = api.product_finder(product_params)
                if not asin_list:
                    print(f"No ASINs for keyword '{keyword}' and brand '{brand}'.")
                    continue

                # ✅ Query only one ASIN
                asin = asin_list[0]
                time.sleep(4)  # 🕒 Throttle per brand request

                raw_products = api.query(
                    [asin],
                    rating=True,
                    buybox=False,
                    update=False,
                    history=True,
                )

                product = raw_products[0] if raw_products and isinstance(raw_products[0], dict) else None
                if not product:
                    continue

                asin = product.get('asin', 'N/A')
                title = product.get('title', 'N/A')
                brand_actual = product.get('brand', 'N/A')

                # NEW price
                price_data = product.get('data', {}).get('NEW', [])
                latest_price = next((p for p in reversed(price_data) if p and not np.isnan(p)), None)
                formatted_price = f"${latest_price:.2f}" if latest_price else "N/A"

                # Rating
                rating_data = product.get('csv', [[], []])[16]
                latest_rating = rating_data[-1] / 10 if rating_data and rating_data[-1] != -1 else "N/A"

                # Review Count
                review_data = product.get('csv', [[], []])[17]
                latest_review_count = review_data[-1] if review_data and review_data[-1] != -1 else "N/A"

                # Suffix to allow multiple products per row
                suffix = f"({column_suffix})" if column_suffix > 1 else ""
                product_row[f"Brand{suffix}"] = brand
                product_row[f"ASIN{suffix}"] = asin
                product_row[f"Title{suffix}"] = title
                product_row[f"Price{suffix}"] = formatted_price
                product_row[f"Rating{suffix}"] = latest_rating
                product_row[f"Review Count{suffix}"] = latest_review_count

                print(f"[✓] {keyword} | {brand} | ASIN: {asin} | Price: {formatted_price} | "
                      f"Rating: {latest_rating} | Reviews: {latest_review_count}")

                column_suffix += 1

            except RuntimeError as e:
                print(f"[✗] API Error for query '{keyword}' and brand '{brand}': {e}")
                continue
            

    # Final output
        results.append(product_row)
        time.sleep(3)

    df = pd.DataFrame(results)
    df.to_csv(dfname)

    return pd.DataFrame(results)


In [7]:
queries = pd.read_csv("cleaned_brand_terms.csv")['Product Name']
allowed_brands = ["ARANA","Suspensia"]

In [69]:
second = fetch_products_from_queries(queries.iloc[4:10], allowed_brands)





100%|██████████| 1/1 [00:03<00:00,  3.20s/it]


[✓] X23CJ7188 Suspension Control Arm and Ball Joint Assembly Fits Hyundai Kia Front Left Lower | ARANA | ASIN: B08CV97X67 | Price: $22.99 | Rating: 4.5 | Reviews: 7712






100%|██████████| 1/1 [00:02<00:00,  2.51s/it]


[✓] X23CJ7188 Suspension Control Arm and Ball Joint Assembly Fits Hyundai Kia Front Left Lower | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A






100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


...

[✓] Sets LGP6 Universal Lift Supports Gas Props Springs Struts for Tool Box Lid Top RV Camper Marine Beds Motorhome Doors | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A






100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


[✓] Hood Lift Support Compatible with Ford F Raptor | ARANA | ASIN: B08CV97X67 | Price: $22.99 | Rating: 4.5 | Reviews: 7712






100%|██████████| 1/1 [00:02<00:00,  2.15s/it]

[✓] Hood Lift Support Compatible with Ford F Raptor | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


In [70]:
second.to_csv("products2.csv", index=False)

In [71]:
second

,Search Query,Brand,ASIN,Title,Price,Rating,Review Count,Brand(2),ASIN(2),Title(2),Price(2),Rating(2),Review Count(2)
0,X23CJ7188 Suspension Control Arm and Ball Join...,ARANA,B08CV97X67,"ARANA Gas Struts 15 inch 60 Lbs, ml13-60 Gas L...",$22.99,4.5,7712,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A
1,X23CJ7187 Suspension Control Arm and Ball Join...,ARANA,B08CV97X67,"ARANA Gas Struts 15 inch 60 Lbs, ml13-60 Gas L...",$22.99,4.5,7712,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A
2,X18CJ1696 Suspension Control Arm and Ball Join...,ARANA,B07CGKD9XP,"ARANA Gas Struts Lift Support 15 inch 24 Lbs, ...",$19.59,4.5,7721,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A
3,X07SL1199 Suspension Stabilizer Bar Link Fits ...,ARANA,B07CGKD9XP,"ARANA Gas Struts Lift Support 15 inch 24 Lbs, ...",$19.59,4.5,7721,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A
4,Sets LGP6 Universal Lift Supports Gas Props Sp...,ARANA,B08CV97X67,"ARANA Gas Struts 15 inch 60 Lbs, ml13-60 Gas L...",$22.99,4.5,7712,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A
5,Hood Lift Support Compatible with Ford F Raptor,ARANA,B08CV97X67,"ARANA Gas Struts 15 inch 60 Lbs, ml13-60 Gas L...",$22.99,4.5,7712,Suspensia,B0C75WP31W,Suspensia X14CJ1539 Suspension Control Arm and...,$48.92,N/A,N/A


In [72]:
third = fetch_products_from_queries(queries.iloc[10:90], allowed_brands)





100%|██████████| 1/1 [00:02<00:00,  2.34s/it]


[✓] Seat Lift Support Compatible with Suzuki Burgman AN650 AN650A Door Motorcycle W/Stock Seat | ARANA | ASIN: B08CV97X67 | Price: $22.99 | Rating: 4.5 | Reviews: 7712






100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


[✓] Seat Lift Support Compatible with Suzuki Burgman AN650 AN650A Door Motorcycle W/Stock Seat | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A






100%|██████████| 1/1 [00:02<00:00,  2.99s/it]


...

[✓] Rear Trunk Lid Lift Supports To Cadillac Sts | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A




Waiting 55 seconds for additional tokens


100%|██████████| 1/1 [00:58<00:00, 58.23s/it]


[✓] Trunk Lid Lift Supports Fits to Buick Century to Buick Regal to Oldsmobile Intrigue Sedan Without Spoiler | ARANA | ASIN: B0D4YSRHJT | Price: $41.99 | Rating: 4.5 | Reviews: 7687




Waiting 28 seconds for additional tokens


100%|██████████| 1/1 [00:31<00:00, 31.09s/it]

[✓] Trunk Lid Lift Supports Fits to Buick Century to Buick Regal to Oldsmobile Intrigue Sedan Without Spoiler | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


In [73]:
third.to_csv("products3.csv", index=False)

In [ ]:
fourth = fetch_products_from_queries(queries.iloc[90:150], allowed_brands,dfname='Products4.csv')

100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


[✓] Front Hood Lift Supports Land Rover Range Rover | ARANA | ASIN: B0BN6382CS | Price: $38.99 | Rating: 4.5 | Reviews: 7725


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


[✓] Front Hood Lift Supports Land Rover Range Rover | Suspensia | ASIN: B0C75TMPV1 | Price: $48.90 | Rating: 5.0 | Reviews: 1


100%|██████████| 1/1 [00:01<00:00,  1.14s/it]


...

100%|██████████| 1/1 [00:00<00:00,  1.59it/s]


[✓] Hood Lift Supports Fits To Kia Carnival / To Kia Sedona | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


[✓] Rear Trunk Lid Lift Supports to Hyundai Azera | ARANA | ASIN: B0D2HNTFC7 | Price: $40.99 | Rating: 4.5 | Reviews: 7712


  0%|          | 0/1 [00:00<?, ?it/s]Waiting 206832 seconds for additional tokens


In [ ]:
fifth = fetch_products_from_queries(queries.iloc[140:180], allowed_brands,dfname='Products5.csv')

100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


[✓] Hood Lift Supports Fits To Kia Carnival / To Kia Sedona | ARANA | ASIN: B0D2HNTFC7 | Price: $40.99 | Rating: 4.5 | Reviews: 7712


100%|██████████| 1/1 [00:02<00:00,  2.18s/it]


[✓] Hood Lift Supports Fits To Kia Carnival / To Kia Sedona | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


...

100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


[✓] Hood Lift Supports Fits to Nissan Maxima | ARANA | ASIN: B0D2HNTFC7 | Price: $40.99 | Rating: 4.5 | Reviews: 7712


100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


[✓] Hood Lift Supports Fits to Nissan Maxima | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


  0%|          | 0/1 [00:00<?, ?it/s]Waiting 206818 seconds for additional tokens


In [8]:
sixth = fetch_products_from_queries(queries.iloc[144:180], allowed_brands,dfname='Products6.csv')

100%|██████████| 1/1 [00:03<00:00,  3.21s/it]


[✓] Hood Lift Supports to Land Rover Discovery to Land Rover Lr3 to Land Rover Lr4 to Land Rover Range Rover Sport | ARANA | ASIN: B0D2HNTFC7 | Price: $40.99 | Rating: 4.5 | Reviews: 7712


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


[✓] Hood Lift Supports to Land Rover Discovery to Land Rover Lr3 to Land Rover Lr4 to Land Rover Range Rover Sport | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:01<00:00,  1.65s/it]


...

[✓] Rear Hatch Lift Supports To Dodge Shadow To Plymouth Sundance With Spoiler | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


[✓] Front Hood Lift Supports To Hyundai Santa Fe | ARANA | ASIN: B08CV97X67 | Price: $22.99 | Rating: 4.5 | Reviews: 7712


100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


[✓] Front Hood Lift Supports To Hyundai Santa Fe | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


In [ ]:
sixth = fetch_products_from_queries(queries.iloc[180:], allowed_brands,dfname='Products7.csv')

100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


[✓] Rear Window Glass Lift Supports ." Doors Only With Rear Drop Gate Only Replaces: To Gmc Jimmy To Chevrolet Blazer | ARANA | ASIN: B08CV97X67 | Price: $22.99 | Rating: 4.5 | Reviews: 7712


100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


[✓] Rear Window Glass Lift Supports ." Doors Only With Rear Drop Gate Only Replaces: To Gmc Jimmy To Chevrolet Blazer | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:01<00:00,  1.55s/it]


...

[✓] Rear Window Lift Supports Fits To Nissan Pathfinder To Infiniti Qx4 | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


[✓] Rear Trunk Lid Lift Supports to Volvo C30 | ARANA | ASIN: B0BN6382CS | Price: $38.99 | Rating: 4.5 | Reviews: 7725


  0%|          | 0/1 [00:00<?, ?it/s]Waiting 64 seconds for additional tokens


ReadTimeout: HTTPSConnectionPool(host='api.keepa.com', port=443): Read timed out. (read timeout=10.0)

In [ ]:

eightth = fetch_products_from_queries(queries.iloc[212:], allowed_brands,dfname='Product8.csv')

100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


[✓] Rear Trunk Lid Lift Supports to Volvo C30 | ARANA | ASIN: B0BN6382CS | Price: $38.99 | Rating: 4.5 | Reviews: 7725


100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


[✓] Rear Trunk Lid Lift Supports to Volvo C30 | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:00<00:00,  1.27it/s]


...

[✓] Liftgate Lift Supports Hyundai Tucson W O Power | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


[✓] Hood Lift Supports Compatible With Lexus Ls460 Ls600H | ARANA | ASIN: B09DG24HCL | Price: N/A | Rating: N/A | Reviews: N/A


Waiting 60 seconds for additional tokens


ConnectionError: HTTPSConnectionPool(host='api.keepa.com', port=443): Max retries exceeded with url: /token/?key=a1g603bd0m13rt71rksm50t36ane60qc8kvioi1ivo3b4sf1qkbj0net036tt6ql (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001D336EC9730>: Failed to resolve 'api.keepa.com' ([Errno 11001] getaddrinfo failed)"))

In [11]:

ningth = fetch_products_from_queries(queries.iloc[337:400], allowed_brands,dfname='Product9.csv')



100%|██████████| 1/1 [00:01<00:00,  1.45s/it]


[✓] Hood Lift Supports Compatible With Lexus Ls460 Ls600H | ARANA | ASIN: B09DG24HCL | Price: N/A | Rating: N/A | Reviews: N/A




100%|██████████| 1/1 [00:01<00:00,  1.93s/it]


[✓] Hood Lift Supports Compatible With Lexus Ls460 Ls600H | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A




100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


...

[✓] Rear Liftgate To Ford Explorer Without Power Liftgate | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A




100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


[✓] Rear Gate Lift Supports Fits Kia and Hyundai with Powered Liftgate Model Years Below | ARANA | ASIN: B0BN674WNV | Price: $27.99 | Rating: 4.5 | Reviews: 7721



Waiting 66 seconds for additional tokens

100%|██████████| 1/1 [01:07<00:00, 67.16s/it]


[✓] Rear Gate Lift Supports Fits Kia and Hyundai with Powered Liftgate Model Years Below | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A


In [ ]:
ningth = fetch_products_from_queries(queries.iloc[400:], allowed_brands,dfname='Product10.csv')



100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


[✓] Liftgate Lift Supports Compatible With: To Infiniti QX56 / To Nissan Armada Power Liftgate | ARANA | ASIN: B08B3Y7Y7W | Price: $27.99 | Rating: 4.5 | Reviews: 7721




100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


[✓] Liftgate Lift Supports Compatible With: To Infiniti QX56 / To Nissan Armada Power Liftgate | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A




100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


...


Waiting 34 seconds for additional tokens

100%|██████████| 1/1 [00:35<00:00, 35.34s/it]


[✓] Front Hood Lift Supports Cadillac Xts | Suspensia | ASIN: B0C75WP31W | Price: $48.92 | Rating: N/A | Reviews: N/A




100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


[✓] Rear Gate Lift Supports Fits To Toyota Sequoia With Power Gate | ARANA | ASIN: B0D2HNTFC7 | Price: $40.99 | Rating: 4.5 | Reviews: 7712



Waiting 66 seconds for additional tokens
